In [1]:
import networkx as nx
import heapq
import collections

In [2]:
def eppstein_sample_graph():
    # Example from
    # Finding the k Shortest Paths, David Eppstein, March 31, 1997
    B = [
        (0,1,2),
        (1,2,20),
        (2,3,14),

        (0,4,13),
        (1,5,27),
        (2,6,14),
        (3,7,15),

        (4,5,9),
        (5,6,10),
        (6,7,25),

        (4,8,15),
        (5,9,20),
        (6,10,12),
        (7,11,7),

        (8,9,18),
        (9,10,8),
        (10,11,11)
    ]

    G = nx.DiGraph()
    for u, v, w in B:
        G.add_edge(u, v, weight=w)
    return G

In [3]:
Node = collections.namedtuple('Node', ['cost', 'id', 'edge'])


def pairwise(seq):
    """iterate over each adjacent pair of elements in a list
    """
    for i in range(len(seq)-1):
        yield (seq[i], seq[i+1])


def single_target_dijkstra(G, tgt):
    """Run a single target Dijkstra search 
    """
    return nx.single_source_dijkstra(G.reverse(), tgt)


def shortest_path_tree(G, tgt):
    """Generate a shortest path tree from a weighted digraph
    """
    # this is inefficient. paper reports is should be doable in O(m + n log n)
    dist, paths = single_target_dijkstra(G, tgt)
    pathmap = {}
    for k, v in paths.items():
        pathmap[k] = list(reversed(v))

    T = nx.DiGraph()
    for n in G.nodes:
        T.add_node(n, weight=dist[n])
    for n in G.nodes:
        for p0, p1 in pairwise(paths[n]):
            T.add_edge(p1, p0)
    return T, pathmap


def delta(G, T, e):
    """The sidetrack cost incurred by following an edge
    """
    # see lemma 1
    if e in T.edges:
        # already shortest path, no cost
        return 0
    else:
        # sidetrack, cost in the original graph, less 
        # the difference in cost between the endpoints
        l_e = G.edges[e]['weight']
        d_head_e_t = T.node[e[1]]['weight']
        d_tail_e_t = T.node[e[0]]['weight']
        res = l_e + d_head_e_t - d_tail_e_t
        return res


def sidetrack_graph(G, T):
    # reformulate the graph and the shortest path tree 
    res = nx.DiGraph()
    for n in G.nodes:
        res.add_node(n)
    for e in G.edges:
        res.add_edge(e[0], e[1], weight=delta(G, T, e))
    return res


def apply_sidetrack(s, t, pathmap, shortcut):
    # apply a sidetrack to a shortest path
    lst = [ s ] 
    while s!=t:
        applied = False
        for su, sv in shortcut:
            if s == su:
                applied = True
                s = sv
                break
        if not applied:
            s = pathmap[s][1]
        lst.append(s)
    return lst


def k_best_sidetrack(G, T, pathmap, s, t, k):
    # find the k-lowest cost sidetracks
    S = sidetrack_graph(G, T)
    Q = []
    heapq.heappush(Q, Node(cost=0, id=s, edge=[]))
    for i in range(k):
        if not Q:
            break
        n = heapq.heappop(Q)
        yield n.edge
        for u, v in pairwise(pathmap[n.id]):
            for w in [ i for i in S.adj[u] if i != v]:
                e = (u, w)
                newnode = Node(cost=n.cost + S.edges[e]['weight'], id=w, edge=n.edge + [e])
                heapq.heappush(Q, newnode)


def k_shortest_paths(G, s, t, k):
    T, pathmap = shortest_path_tree(G, t)
    for sidetrack in k_best_sidetrack(G, T, pathmap, s, t, k):
        pth = apply_sidetrack(s, t, pathmap, sidetrack)
        yield pth


In [4]:
G = eppstein_sample_graph()
for edges in k_shortest_paths(G, 0, 11, 100):
    pedges = list(pairwise(edges))
    cost   = sum([G.edges[e]['weight'] for e in pedges])
    print(edges, cost)

[0, 4, 5, 6, 10, 11] 55
[0, 1, 2, 3, 7, 11] 58
[0, 1, 2, 6, 10, 11] 59
[0, 4, 5, 9, 10, 11] 61
[0, 1, 5, 6, 10, 11] 62
[0, 4, 5, 6, 7, 11] 64
[0, 4, 8, 9, 10, 11] 65
[0, 1, 2, 6, 7, 11] 68
[0, 1, 5, 9, 10, 11] 68
[0, 1, 5, 6, 7, 11] 71
